In [1]:
import os
os.chdir("/home/017534556/projects/ebay-comp-2024")

In [2]:
from src.utils.helper import read_config

from pathlib import Path
import numpy as np

from src.utils.helper import read_config
from src.utils.submission import (
    db_connect,
    table_exists,
)
import pandas as pd

import sqlite3

import matplotlib.pyplot as plt
import seaborn
from sentence_transformers import SentenceTransformer

from tqdm import tqdm

from src.utils.evaluation import eval_fbeta, pre_process
from src.utils.submission import list_all_tables, read_table

seaborn.set_style(style="whitegrid")

/home/017534556/pyenvs/ebay_oct25_hpc1/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
# config_path = "src/configs/inference_v2.yaml"
config_path = "src/configs/zeroshot_quiz.yaml"

In [4]:
cfg = read_config(config_path)

In [5]:
sub_fn = f"{os.path.basename(config_path).split('.')[0]}_{cfg['submission_table_name']}.csv.gz"
sub_fn

'zeroshot_quiz_zeroshot_nov3_groq_gemma2_9b_cot_quiz.csv.gz'

In [6]:
conn = db_connect(Path(cfg["DB_ROOT"]) / "submissions_bak.db")

if not table_exists(conn, cfg["submission_table_name"]):
    raise Warning("Table does not exist, check the table name")

In [7]:
cursor = conn.cursor()

In [8]:
cursor.execute(f"SELECT * FROM {cfg['submission_table_name']}")
rows = [x[1:] for x in cursor.fetchall()]

In [9]:
df = pd.DataFrame(rows, columns=["RECORD_ID", "FTMNT_YEAR", "FTMNT_MAKE", "FTMNT_MODEL"])

In [10]:
# include rows only with entropy <= 0.5
# df = df[df["ENTROPY"] <= 0.55]

In [11]:
df[df["RECORD_ID"].between(5000, 29999)]

,RECORD_ID,FTMNT_YEAR,FTMNT_MAKE,FTMNT_MODEL
0,5004,2015,mazda,3
1,5000,2005,dodge,300
2,5000,2006,dodge,300
3,5000,2007,dodge,300
4,5000,2008,dodge,300
...,...,...,...,...
230760,29998,2010,audi,s8
230761,29998,2011,audi,s8
230762,29999,1982,toyota,celica
230763,29999,1983,toyota,celica


In [12]:
df["RECORD_ID"].unique().shape

(20939,)

In [13]:
df.isna().sum()

RECORD_ID      0
FTMNT_YEAR     0
FTMNT_MAKE     0
FTMNT_MODEL    0
dtype: int64

In [14]:
df = df[df["RECORD_ID"].between(5000, 29999)]
# df[df.select_dtypes(include=['object']).columns] = df.select_dtypes(include=['object']).apply(lambda x: x.str.strip())
df.replace("", np.nan, inplace=True)
df.replace("N/A", np.nan, inplace=True)
df.replace("NA", np.nan, inplace=True)
df.replace("na", np.nan, inplace=True)
df.replace("n/a", np.nan, inplace=True)
df.replace("None", np.nan, inplace=True)
df.replace("none", np.nan, inplace=True)
df.replace("NaN", np.nan, inplace=True)
df.replace("nan", np.nan, inplace=True)
df.replace("null", np.nan, inplace=True)
df.replace("NULL", np.nan, inplace=True)
df.dropna(how='any', inplace=True)
# Q: ValueError: Cannot mask with non-boolean array containing NA / NaN values

# A: df = df[df["FTMNT_YEAR"].str.isnumeric()] will raise the error because of NaN values in the column

# if record id has values less than 5000 or more than 29999, drop them


In [15]:
assert df.isna().sum().sum() == 0, "WARNING, NAN VALUES IN THE DATAFRAME"

In [16]:
df["RECORD_ID"].unique().shape

(20931,)

In [17]:
# only inlcude the columns that are needed
df = df[["RECORD_ID", "FTMNT_YEAR", "FTMNT_MAKE", "FTMNT_MODEL"]]

In [18]:
class EbayMMcoder:
    def __init__(self, model_name="multi-qa-mpnet-base-dot-v1"):
        self.model = SentenceTransformer(model_name)
    
    def encode(self, make_models):
        self.make_models = make_models
        self.pairwise_embeddings = self.model.encode([f"{make}+{model}" for make, model in [(x.lower(), y.lower()) for x, y in make_models]])
        
    
    def similar_make(self, make):
        score, indices = self.model.similarity(self.model.encode([make]), self.makes_embeddings).topk(1)
        return self.makes[indices[0]]
    
    def similar_model(self, model):
        score, indices = self.model.similarity(self.model.encode([model]), self.models_embeddings).topk(1)
        return self.models[indices[0]]

    def get_make_model(self, makemodel):
        score, indices = self.model.similarity(self.model.encode(makemodel), self.pairwise_embeddings).topk(1)
        return [self.make_models[x] for x in indices.flatten()]

mmcoder = EbayMMcoder()

In [19]:
conn = sqlite3.connect(database="db/submissions_bak.db")
cursor = conn.cursor()
cursor.execute(
f"""SELECT RECORD_ID, FTMNT_YEAR, FTMNT_MAKE, FTMNT_MODEL FROM ground_truth"""
)
gt_results= [x for x in cursor.fetchall()]
conn.close()
gt_record_ids = [x[0] for x in gt_results]

In [20]:
car_makes = [x[2] for x in gt_results]
car_models = [x[3] for x in gt_results]

In [21]:
car_make_models = list(set([(x, y) for x, y in zip(car_makes, car_models)]))

In [22]:
mmcoder.encode(car_make_models)

In [23]:
mmcoder.get_make_model(["chevy+camaro", "ford+mustang", "Mercedes+500"])

[('Chevrolet', 'Camaro'), ('Ford', 'Mustang'), ('Mercedes-Benz', '500SEL')]

In [24]:
mmcoder.get_make_model("American Motors+ classic")

[('Chevrolet', 'Classic')]

In [25]:
class ProcessSubmission:
    def __init__(self, df):
        self.df = df
        self.records = df.to_numpy().tolist()
        self.current_count = len(self.records)
        self.new_count = 0
    
    def split_years(self):
        for record in self.records:
            year = record[1]
            start_year, end_year = None, None
            
            if isinstance(year, float):
                continue
            
            if not isinstance(year, int) and "-" in year:
                year = year.split("-")
                if not year[0].isnumeric() or not year[1].isnumeric():
                    continue
                    
                start_year, end_year = int(year[0]), int(year[1])
                for i in range(start_year, end_year+1):
                    self.records.append([record[0], i, record[2], record[3]])
                    self.new_count += 1

            if len(str(year)) == 2 and str(year).isnumeric():
                year = int(f"20{year}")
                self.records.append([record[0], year, record[2], record[3]])
                self.new_count += 1 

    def remove_str_years(self):
        # copy of self.records
        clone_records = []
        for record in self.records:
            year = record[1]
            len_year = len(str(year))
            if isinstance(year, str):
                self.new_count -= 1
                continue
            if isinstance(year, int):
                if (len_year < 4 or len_year > 4):
                    self.new_count -= 1
                    continue
                else:
                    if year >= 2025 or year <= 1885:
                        self.new_count -= 1
                        continue
                clone_records.append(record)
                        
                        
            
        self.records = clone_records
    
    def correct_make_model(self, bsz=128):
        new_records = []
        i = 0
        for i in tqdm(range(0, len(self.records), bsz)):
            batch = []
            data = []
            for record in self.records[i:i+bsz]:
                batch.append(f"{record[2]}+{record[3]}")
                data.append((record[0], record[1]))
            make_models = mmcoder.get_make_model(batch)
            for idx, (make, model) in enumerate(make_models):
                new_records.append([data[idx][0], data[idx][1], make, model])
        self.records = new_records
    
    def convert_to_df(self):
        return pd.DataFrame(self.records, columns=["RECORD_ID", "FTMNT_YEAR", "FTMNT_MAKE", "FTMNT_MODEL"])


In [26]:
p = ProcessSubmission(df)

In [27]:
p.split_years()
p.remove_str_years()

In [28]:
p.current_count

230682

In [29]:
p.correct_make_model()

  0%|          | 0/1800 [00:00<?, ?it/s]

100%|██████████| 1800/1800 [01:08<00:00, 26.32it/s]


In [31]:
p.new_count

-375

In [32]:
p.convert_to_df()

,RECORD_ID,FTMNT_YEAR,FTMNT_MAKE,FTMNT_MODEL
0,5004,2015,Mazda,3
1,5000,2005,Dodge,400
2,5000,2006,Dodge,400
3,5000,2007,Dodge,400
4,5000,2008,Dodge,400
...,...,...,...,...
230287,6079,2013,Volvo,760
230288,6079,2010,Volvo,C30
230289,6079,2011,Volvo,C30
230290,6079,2012,Volvo,C30


In [34]:
df = p.convert_to_df()

In [35]:
df["FTMNT_YEAR"].unique()

array([2015, 2005, 2006, 2007, 2008, 2009, 2010, 1986, 1987, 1988, 1989,
       1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 1980,
       1981, 1982, 1983, 1984, 2011, 2012, 2013, 2016, 2002, 2003, 2004,
       2001, 2000, 2018, 2019, 2020, 2021, 2022, 2023, 1985, 1970, 1971,
       1972, 1973, 2014, 2017, 1974, 1975, 1976, 1960, 1961, 1962, 1963,
       1964, 1957, 1958, 1977, 1978, 1979, 1965, 1966, 1967, 1968, 1969,
       1959, 1940, 1953, 1954, 1955, 1952, 1956, 1951, 1949, 2024, 1950,
       1946, 1947, 1948, 1937, 1928, 1932, 1942, 1943, 1944, 1945, 1933,
       1934, 1935, 1936, 1929, 1930, 1931, 1941, 1908, 1927, 1939, 1938,
       1925, 1926, 1920, 1917, 1918, 1919, 1921, 1922, 1923, 1924])

In [36]:
sub_fn

'zeroshot_quiz_zeroshot_nov3_groq_gemma2_9b_cot_quiz.csv.gz'

In [37]:
base_name = os.path.basename(sub_fn).split(".")[0]

In [38]:
df.head()

,RECORD_ID,FTMNT_YEAR,FTMNT_MAKE,FTMNT_MODEL
0,5004,2015,Mazda,3
1,5000,2005,Dodge,400
2,5000,2006,Dodge,400
3,5000,2007,Dodge,400
4,5000,2008,Dodge,400


In [42]:
df.to_csv(f"/home/017534556/projects/ebay-comp-2024/submissions/{sub_fn}", index=False, compression="gzip")

In [418]:
# df.to_csv(f"/home/017534556/projects/ebay-comp-2024/submissions/{base_name+'_entropy_055'+'.csv.gz'}", index=False, compression="gzip")

In [43]:
!zcat /home/017534556/projects/ebay-comp-2024/submissions/{sub_fn} > /home/017534556/projects/ebay-comp-2024/submissions/{sub_fn.split('.')[0]}.csv

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [45]:
new_df = pd.read_csv("/home/017534556/projects/ebay-comp-2024/submissions/zeroshot_quiz_zeroshot_nov3_groq_gemma2_9b_cot_quiz.csv")

In [46]:
new_df[new_df.isna().any(axis=1)]

,RECORD_ID,FTMNT_YEAR,FTMNT_MAKE,FTMNT_MODEL


In [47]:
record_ids_to_check = new_df[new_df.isna().any(axis=1)]["RECORD_ID"].unique()
record_ids_to_check

array([], dtype=int64)

In [48]:
# view nan data rows
df[df["RECORD_ID"].isin(record_ids_to_check)]


,RECORD_ID,FTMNT_YEAR,FTMNT_MAKE,FTMNT_MODEL
